This notebook was created with source code from https://github.com/chambbj/pdal-notebook/blob/master/notebooks/DMP.ipynb, based off [1] D. Mongus, N. Lukac, and B. Zalik, “Ground and building extraction from LiDAR data based on differential morphological profiles and locally fitted surfaces,” ISPRS J. Photogramm. Remote Sens., vol. 93, no. January, pp. 145–156, 2014.

Changes in this algorithm: k = 0 (so, removal of gstar matrix), b = -2, manually set hres (relative to density), added PMF at end to clean up

In [ ]:
from scipy import ndimage, signal, spatial
from scipy.ndimage import morphology

import numpy as np
import pandas as pd
import pdal

In [ ]:
#if adding resolution as a parameter:
#determine if goal resolution is feasible? (of course this is contingent upon local machine specs and the user's patience)


data_in = "E:/SFSU/UAS/panca_721000_4426500.laz"

#read in in full res .las , subsampled with Poisson, change radius to reach desired density
p = pdal.Reader.las(data_in).pipeline() | pdal.Filter.sample(radius=0.5).pipeline()
n_points = p.execute()
f'Pipeline selected {n_points} points'

In [ ]:
#create a one dimensional array from the "Classification" column
cls = p.arrays[0]['Classification']
#set the array to all ones
cls.fill(1)

In [ ]:
#convert X,Y, and Z data to a pandas dataframe
df3D = pd.DataFrame(p.arrays[0], columns=['X','Y','Z'])

In [ ]:
#define variables
S = 20
#k = 0.000
n = 0.1
b = -0.2

In [ ]:
#can't find any documentation on .ptp(), but it must be the x and y length for computing the area
density = n_points / (p.arrays[0]['Y'].ptp() * p.arrays[0]['X'].ptp())
#hres = 1. / density
#the above method for calculating hres is not suggested
#for densities around 1pt/m2 try hres = 1, for densities around 15pt/m2 try hres range: 0.25 to 0.5 (more tests to come)
hres = 0.25

In [ ]:
print("Point cloud density estimated as", density, "pts/m^2. Processing at", hres, "m resolution.")

In [ ]:
#np.ogrid "open-grid", creates a way to index the matrix (access pixels/pts) hres is the step
xi = np.ogrid[p.arrays[0]['X'].min():p.arrays[0]['X'].max():hres]
yi = np.ogrid[p.arrays[0]['Y'].min():p.arrays[0]['Y'].max():hres]

In [ ]:
#np.digitize allocates points to bins and then bins are grouped in the df
bins = df3D.groupby([np.digitize(p.arrays[0]['X'], xi), np.digitize(p.arrays[0]['Y'], yi)])

In [ ]:
zmins = bins.Z.min() #collects the lowest point in each bin
cz = np.empty((yi.size, xi.size)) #create empty 2d array 
cz.fill(np.nan) #fill 2d array with nan
for name, val in zmins.iteritems():
    cz[name[1]-1, name[0]-1] = val #adding coordinates to lowest points only(not sure why -1 is used here)

In [ ]:
def idw(data):
    # Find indices of the ground returns, i.e., anything that is not a nan, and create a KD-tree.
    # We will search this tree when looking for nearest neighbors to perform the interpolation.
    valid = np.argwhere(~np.isnan(data))
    tree = spatial.cKDTree(valid)
    
    # Now find indices of the non-ground returns, as indicated by nan values. We will interpolate
    # at these locations.
    nans = np.argwhere(np.isnan(data))    
    for row in nans:
        d, idx = tree.query(row, k=12) #k = number of nearest neighbors
        d = np.power(d, -2) #each item in d raised to its reciprocated power (basis of idw) the value "r" also defines the smoothness of the interpolation
        v = data[valid[idx, 0], valid[idx, 1]] 
        data[row[0], row[1]] = np.inner(v, d)/np.sum(d) #nans are replaced with the result of (v * d)/sum(d)
        
    return data

In [ ]:
cz = idw(cz)

In [ ]:
#create an initial diamond 2,1 and enlarge it 11 times = 23x,23y
struct = ndimage.iterate_structure(ndimage.generate_binary_structure(2, 1), 11).astype(int)
opened = morphology.grey_opening(cz, structure=struct)

In [ ]:
#create another diamond (2,1) and enlarge it 9 times = 19x,19y
struct = ndimage.iterate_structure(ndimage.generate_binary_structure(2, 1), 9).astype(int)
closed = morphology.grey_closing(opened, structure=struct)

In [ ]:
#removing low outliers: if any pt in cz is >= 1 meter below the surface of closed then it is set to the 
#closed surface value
lowx, lowy = np.where((closed - cz) >= 1.0) 
cz[lowx, lowy] = closed[lowx, lowy]

In [ ]:
stdev = 14
#product of two guassian arrays with the max normalized to 1, size/window = 113
G = np.outer(signal.gaussian(113,stdev), signal.gaussian(113,stdev))
#fast fourier transform convolution, matrix is padded at 2*stdev
low = signal.fftconvolve(np.pad(cz,2*stdev,'edge'), G, mode='same')[2*stdev:-2*stdev,2*stdev:-2*stdev]/1000.

In [ ]:
high = cz - low

In [ ]:
erosions = []
granulometry = []
erosions.append(morphology.grey_erosion(high, size=3))
for scale in range(1, S):
    erosions.append(morphology.grey_erosion(erosions[scale-1], size=3))
for scale in range(1, S+1):
    granulometry.append(morphology.grey_dilation(erosions[scale-1], size=2*scale+1))

In [ ]:
out = []
for i in range(1, len(granulometry)):
    out.append(granulometry[i-1]-granulometry[i])

In [ ]:
gprime = np.maximum.reduce(out)
xs, ys = out[0].shape
#gstar = np.zeros((xs,ys))
gplus = np.zeros((xs,ys))
for ii in range(0,xs):
    for jj in range(0,ys):
        for kk in range(0,len(out)):
            if out[kk][ii,jj] < gprime[ii,jj]:
                gplus[ii,jj] += out[kk][ii,jj]
            if out[kk][ii,jj] == gprime[ii,jj]:
                gplus[ii,jj] += out[kk][ii,jj]
                #gstar[ii,jj] = kk
                break

In [ ]:
#T = k * gstar + n
Sg = gprime < n

In [ ]:
F = cz.copy()
F[np.where(Sg==0)] = np.nan

In [ ]:
G = idw(F)

In [ ]:
struct = ndimage.iterate_structure(ndimage.generate_binary_structure(2, 1), 1).astype(int)
gradDTM = morphology.grey_dilation(G, structure=struct)

In [ ]:
xbins = np.digitize(df3D.X, xi)
ybins = np.digitize(df3D.Y, yi)
#nonground = np.where(df3D.Z >= gradDTM[ybins-1, xbins-1]+b)
ground = np.where(df3D.Z < gradDTM[ybins-1, xbins-1]+b)

In [ ]:
cls[ground] = 2 #set ground points to 2
len(cls[ground]) #number of ground points

In [ ]:
output = p.arrays[0]
output['Classification'] =cls

In [ ]:
#include only ground points
pipeline = pdal.Filter.range(limits="Classification[2:2]").pipeline(output)
print(pipeline.execute())

In [ ]:
#default Progressive morphological filter stacked to catch stragglers (havent'tested with alt parameters)
#need to test with alt smrf
pmf_arr = pipeline.arrays[0]
pipeline = pdal.Filter.pmf().pipeline(pmf_arr) | pdal.Filter.range(limits="Classification[2:2]").pipeline()
print(pipeline.execute())

In [ ]:
#writout las file with ground points only 
final_out = pipeline.arrays[0]
pipeline = pdal.Writer.las(filename="ground_only.las",).pipeline(final_out)
print(pipeline.execute())